In [0]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path = "/content/drive/My Drive/NLP-Exercises-master/2.2-topic-modeling/abcnews-date-text.csv"

In [0]:
data = pd.read_csv(path, error_bad_lines = False)
data_text = data[:300000][['headline_text']]
data_text['index'] = data_text.index

documents = data_text

In [0]:
documents.head()

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


In [0]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np

np.random.seed(400)

In [0]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
print(WordNetLemmatizer().lemmatize('ate', pos = 'v'))

eat


In [0]:
stemmer = SnowballStemmer('english')

original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']

singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word': original_words, 'stemmed':singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [0]:
def lemmatize_stemming(text):
  return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos = 'v'))

def preprocess(text):
  result = []
  for token in gensim.utils.simple_preprocess(text):
    if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
      result.append(lemmatize_stemming(token))
  return result 

In [0]:
document_num = 4310
doc_sample = documents[documents['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
  words.append(word)

print(words)
print("\n\nTokenize and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenize and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [0]:
processed_docs = documents['headline_text'].map(preprocess)

In [0]:
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [0]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [0]:
count = 0
for k, v in dictionary.iteritems():
  print(k,v)
  count += 1
  if count > 10 :
    break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [0]:
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=100000)

In [0]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [0]:
bow_corpus[0]

[(0, 1), (1, 1), (2, 1), (3, 1)]

In [0]:
bow_doc_0 = bow_corpus[document_num]

for i in range(len(bow_doc_0)):
  print("Word {} (\"{}\") appears {} time.".format(bow_doc_0[i][0], 
                                                     dictionary[bow_doc_0[i][0]], 
                                                     bow_doc_0[i][1]))

Word 71 ("bushfir") appears 1 time.
Word 107 ("help") appears 1 time.
Word 462 ("rain") appears 1 time.
Word 3530 ("dampen") appears 1 time.


In [0]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [0]:
corpus_tfidf = tfidf[bow_corpus]

In [0]:
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5959813347777092),
 (1, 0.39204529549491984),
 (2, 0.48531419274988147),
 (3, 0.5055461098578569)]


In [0]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes =2, workers=2)

In [0]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.022*"closer" + 0.021*"test" + 0.020*"lead" + 0.017*"talk" + 0.014*"south" + 0.013*"law" + 0.012*"take" + 0.012*"timor" + 0.011*"west" + 0.010*"open"


Topic: 1 
Words: 0.092*"polic" + 0.028*"seek" + 0.025*"investig" + 0.022*"miss" + 0.016*"probe" + 0.015*"search" + 0.013*"region" + 0.011*"offic" + 0.011*"bodi" + 0.011*"park"


Topic: 2 
Words: 0.016*"record" + 0.014*"break" + 0.014*"australia" + 0.013*"look" + 0.012*"rain" + 0.012*"drought" + 0.012*"dead" + 0.012*"price" + 0.010*"sydney" + 0.010*"fall"


Topic: 3 
Words: 0.050*"water" + 0.033*"warn" + 0.015*"industri" + 0.015*"continu" + 0.013*"farmer" + 0.012*"urg" + 0.012*"busi" + 0.012*"begin" + 0.011*"worker" + 0.010*"govt"


Topic: 4 
Words: 0.016*"elect" + 0.016*"iraq" + 0.014*"howard" + 0.014*"deal" + 0.013*"market" + 0.013*"labor" + 0.012*"say" + 0.011*"appeal" + 0.011*"plan" + 0.011*"troop"


Topic: 5 
Words: 0.040*"charg" + 0.034*"court" + 0.033*"face" + 0.023*"kill" + 0.020*"murder" + 0.020*"accus" + 0.020

In [0]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [0]:
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5591250658035278	 
Topic: 0.016*"record" + 0.014*"break" + 0.014*"australia" + 0.013*"look" + 0.012*"rain" + 0.012*"drought" + 0.012*"dead" + 0.012*"price" + 0.010*"sydney" + 0.010*"fall"

Score: 0.2808539867401123	 
Topic: 0.050*"water" + 0.033*"warn" + 0.015*"industri" + 0.015*"continu" + 0.013*"farmer" + 0.012*"urg" + 0.012*"busi" + 0.012*"begin" + 0.011*"worker" + 0.010*"govt"

Score: 0.02000882849097252	 
Topic: 0.092*"polic" + 0.028*"seek" + 0.025*"investig" + 0.022*"miss" + 0.016*"probe" + 0.015*"search" + 0.013*"region" + 0.011*"offic" + 0.011*"bodi" + 0.011*"park"

Score: 0.02000432275235653	 
Topic: 0.054*"govt" + 0.030*"council" + 0.024*"fund" + 0.022*"plan" + 0.021*"urg" + 0.017*"boost" + 0.013*"servic" + 0.012*"health" + 0.012*"defend" + 0.012*"rise"

Score: 0.020003890618681908	 
Topic: 0.018*"return" + 0.017*"hold" + 0.014*"question" + 0.014*"work" + 0.013*"firefight" + 0.013*"resid" + 0.011*"blaze" + 0.010*"rais" + 0.010*"battl" + 0.010*"unit"

Score: 0.020001

In [0]:

unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.4200000762939453	 Topic: 0.022*"closer" + 0.021*"test" + 0.020*"lead" + 0.017*"talk" + 0.014*"south"
Score: 0.2199999839067459	 Topic: 0.018*"return" + 0.017*"hold" + 0.014*"question" + 0.014*"work" + 0.013*"firefight"
Score: 0.21999213099479675	 Topic: 0.040*"charg" + 0.034*"court" + 0.033*"face" + 0.023*"kill" + 0.020*"murder"
Score: 0.02000407502055168	 Topic: 0.038*"crash" + 0.025*"jail" + 0.021*"road" + 0.017*"die" + 0.017*"death"
Score: 0.020003709942102432	 Topic: 0.036*"report" + 0.024*"opposit" + 0.023*"power" + 0.014*"win" + 0.012*"say"
Score: 0.019999999552965164	 Topic: 0.092*"polic" + 0.028*"seek" + 0.025*"investig" + 0.022*"miss" + 0.016*"probe"
Score: 0.019999999552965164	 Topic: 0.016*"record" + 0.014*"break" + 0.014*"australia" + 0.013*"look" + 0.012*"rain"
Score: 0.019999999552965164	 Topic: 0.050*"water" + 0.033*"warn" + 0.015*"industri" + 0.015*"continu" + 0.013*"farmer"
Score: 0.019999999552965164	 Topic: 0.016*"elect" + 0.016*"iraq" + 0.014*"howard" + 0.0